SCRAPING http://www.indeed.com for average salaries by job title

In [499]:
#import environments
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
import time

In [500]:
# https://splinter.readthedocs.io/en/latest/drivers/chrome.html
!which chromedriver

/usr/local/bin/chromedriver


In [501]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [502]:
url = 'https://www.indeed.com/salaries'
browser.visit(url)
time.sleep(1)

KeyboardInterrupt: 

In [492]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
#print(soup.prettify())

In [493]:
job_titles = ["data entry clerk", "entry level data analyst",'junior data analyst', 
              'data analyst', 'senior data analyst', 
              'data engineer', 'data scientist', 'senior data scientist', 'lead data scientist'
              'principal data scientist','business analyst','Business intelligence analyst',
              'Business intelligence developer', 'software engineer', 'Senior Software Engineer',
              "Chief information officer", "Business Systems Analyst",
             "Data Warehouse Architect", "Software Architect",
             "Database manager","Senior Data modeler", "Database developer", 
              "Database administrator", "Machine learning engineer", "tableau developer"]

# Test Case - try specific title
#job_titles = ['principal data scientist']

salaries_by_job_title = []
salary_type_by_job_title = []
salary_description = []
source_urls = []
number_of_salaries_reported = []


In [494]:
for job in job_titles:
    avg_salary = 'NaN'
    avg_salary_type = 'NaN'
    avg_salary_summary = 'NaN'
    number_of_salaries = 'NaN'
    try:
        print(f'START: Searching for job title: {job}')
        try:
            print('----Trying to find SEARCH BOX by class type 1--------')
            browser.find_by_id('cmp-salary-search-input').first.fill(job)
        except:
            try:
                print('----Trying to find SEARCH BOX by class type 2--------')
                browser.find_by_id('cmp-salary-search-input-title').first.fill(job)
            except:
                print('----ERROR: Could find search box by either class--------')

        time.sleep(1)
        html = browser.html
        soup = BeautifulSoup(html, 'html.parser')
#         print(soup.prettify())
        
        search_results = soup.find('div', class_='cmp-salary-search-results')
#         print(search_results)

        keywords = search_results.find_all('a', class_='cmp-salary-search-result-title')
        for keyword in keywords:
            print(f'SEARCH BOX Keyword returned: {keyword.text}')
        
        # When search field is filled, it dynamically loads a list of job titles in drop-down list. 
        # We ignore choices by doing a partial text search on title we are looking for and if no
        # match exists, find the last link "popular salaries for <title>" and mouse over, so webpage
        # changes the class to forice it in search.
        
        option = browser.links.find_by_partial_text(job)
        option.mouse_over()
        print(f'Mouse Over completed for option: {option.text}')
        browser.click_link_by_id('cmp-salary-search-submit')
        time.sleep(1)
        print(browser.url)
        browser.visit(browser.url)
        time.sleep(1)
        html = browser.html
        soup = BeautifulSoup(html, 'html.parser')
        #print(soup.prettify())
        try:
            print('-----Trying Results Page type 1-------')
            avg_salary = soup.find('div', class_='sal-agg-nonbase__average-salary-value').text
            avg_salary_type = soup.find('div', class_='sal-agg-nonbase__average-salary-type').text   
            avg_salary_summary = soup.find('h3', class_='sal-agg-nonbase__salary-explain-summary').text
            salaries_by_job_title.append(avg_salary)
            salary_type_by_job_title.append(avg_salary_type)
            salary_description.append(avg_salary_summary)
            source_urls.append(browser.url)
            salaries_reported_string = soup.find('div', class_='sal-agg-nonbase__salary-reported').text
            salaries_reported = salaries_reported_string.split('salaries reported')
            print(salaries_reported)
            number_of_salaries = salaries_reported[0]
            number_of_salaries_reported.append(number_of_salaries)
            print('-----Completed Results Page type 1-------')
        except:
            try:
                print('----- Trying Results Page type 2-------')
                avg_salary = soup.find('strong', class_='cmp-salary-amount').text           
                avg_salary_type = soup.find('div', class_='cmp-sal-salary').text 
                avg_salary_type_array = avg_salary_type.split('per') 
                avg_salary_type = f'per{avg_salary_type_array[1]}'
                avg_salary_summary = soup.find('div', class_='cmp-salary-blurb-text').text  
                salaries_by_job_title.append(avg_salary)
                salary_type_by_job_title.append(avg_salary_type)
                salary_description.append(avg_salary_summary)
                source_urls.append(browser.url)
                salaries_reported_string = soup.find('div', class_='cmp-salary-header-content').text
                salaries_reported = salaries_reported_string.split('Salary estimated from')
                print(salaries_reported)
                number_of_salaries = salaries_reported[1].split(' ')
                number_of_salaries = number_of_salaries[1]
                number_of_salaries_reported.append(number_of_salaries)
                print('----- Completed Results Page type 2-------')
            except:
                print('----- No Salary results found -------')
                avg_salary = 'NaN'
                salaries_by_job_title.append(avg_salary)
                salary_type_by_job_title.append(avg_salary_type)
                salary_description.append(avg_salary_summary)
                source_urls.append(browser.url)    
                number_of_salaries_reported.append(number_of_salaries)
            
        print('RECORD elements inserted in lists')
        print('---------------------------------')
        print(avg_salary)
        print(avg_salary_type)
        print(avg_salary_summary)
        print(number_of_salaries)
        browser.back()
        time.sleep(1)
    except AttributeError as e:
        print(f'ERROR: {e}')

        

START: Searching for job title: data entry clerk
----Trying to find SEARCH BOX by class type 1--------
SEARCH BOX Keyword returned: Data Entry Clerk
Mouse Over completed for option: Popular salaries for data entry clerk
https://www.indeed.com/career/data-entry-clerk/salaries
-----Trying Results Page type 1-------
['21.8k ', ', updated at Aug 28, 2020']
-----Completed Results Page type 1-------
RECORD elements inserted in lists
---------------------------------
$2,247
per month
The average salary for a data entry clerk  is $2,247 per month in the United States. 
21.8k 
START: Searching for job title: entry level data analyst
----Trying to find SEARCH BOX by class type 1--------
SEARCH BOX Keyword returned: Entry Level Data Analyst
Mouse Over completed for option: Popular salaries for entry level data analyst
https://www.indeed.com/salaries/entry%20level%20data%20analyst
-----Trying Results Page type 1-------
----- Trying Results Page type 2-------
['', ' 192 employees, users, and past a

https://www.indeed.com/career/business-intelligence-developer/salaries
-----Trying Results Page type 1-------
['1.7k ', ', updated at Aug 28, 2020']
-----Completed Results Page type 1-------
RECORD elements inserted in lists
---------------------------------
$102,168
per year
The average salary for a business intelligence developer  is $102,168 per year in the United States and  $5,000 cash bonus per year.
1.7k 
START: Searching for job title: software engineer
----Trying to find SEARCH BOX by class type 1--------
SEARCH BOX Keyword returned: Software Engineer
SEARCH BOX Keyword returned: Senior Software Engineer
SEARCH BOX Keyword returned: Software Test Engineer
Mouse Over completed for option: Popular salaries for software engineer
https://www.indeed.com/career/software-engineer/salaries
-----Trying Results Page type 1-------
['37.8k ', ', updated at Aug 28, 2020']
-----Completed Results Page type 1-------
RECORD elements inserted in lists
---------------------------------
$110,169


SEARCH BOX Keyword returned: Machine Learning Engineer
Mouse Over completed for option: Popular salaries for Machine learning engineer
https://www.indeed.com/career/machine-learning-engineer/salaries
-----Trying Results Page type 1-------
['1.6k ', ', updated at Aug 27, 2020']
-----Completed Results Page type 1-------
RECORD elements inserted in lists
---------------------------------
$144,264
per year
The average salary for a machine learning engineer  is $144,264 per year in the United States. 
1.6k 
START: Searching for job title: tableau developer
----Trying to find SEARCH BOX by class type 1--------
SEARCH BOX Keyword returned: Tableau Developer
Mouse Over completed for option: Popular salaries for tableau developer
https://www.indeed.com/salaries/tableau%20developer
-----Trying Results Page type 1-------
----- Trying Results Page type 2-------
['', ' 338 employees, users, and past and present job advertisements on Indeed in the past 36 months.\xa0Last updated: August 27, 2020']
-

In [495]:
browser.quit()

In [496]:
print(len(job_titles), len(salaries_by_job_title), len(salary_type_by_job_title), len(salary_description), 
      len(source_urls))

24 24 24 24 24


In [497]:
salary_table = pd.DataFrame({"Job Title":job_titles,
                             "Avg Salary":salaries_by_job_title,
                             "Salary Type": salary_type_by_job_title,
                             "Number of Salaries Reported": number_of_salaries_reported,
                             "Salary Description": salary_description,
                             "Source URL": source_urls,
                             })
salary_table

,Job Title,Avg Salary,Salary Type,Number of Salaries Reported,Salary Description,Source URL
0,data entry clerk,"$2,247",per month,21.8k,The average salary for a data entry clerk is ...,https://www.indeed.com/career/data-entry-clerk...
1,entry level data analyst,"$59,938",per year,192,The average salary for an Entry Level Data An...,https://www.indeed.com/salaries/entry%20level%...
2,junior data analyst,"$57,236",per year,161,The average salary for a junior data analyst ...,https://www.indeed.com/career/junior-data-anal...
3,data analyst,"$75,121",per year,6k,"The average salary for a data analyst is $75,...",https://www.indeed.com/career/data-analyst/sal...
4,senior data analyst,"$95,984",per year,1.6k,The average salary for a senior data analyst ...,https://www.indeed.com/career/senior-data-anal...
5,data engineer,"$129,421",per year,7.8k,The average salary for a data engineer is $12...,https://www.indeed.com/career/data-engineer/sa...
6,data scientist,"$121,691",per year,5.3k,The average salary for a data scientist is $1...,https://www.indeed.com/career/data-scientist/s...
7,senior data scientist,"$150,981",per year,1.5k,The average salary for a senior data scientist...,https://www.indeed.com/career/senior-data-scie...
8,lead data scientistprincipal data scientist,NaN,per year,NaN,NaN,https://www.indeed.com/salaries/lead%20data%20...
9,business analyst,"$80,067",per year,7.8k,The average salary for a business analyst is ...,https://www.indeed.com/career/business-analyst...


In [498]:
salary_table.to_csv(r'avg_salary_by_title.csv', index = False)